<a href="https://colab.research.google.com/github/Steve-Falkovsky/Hypencoder-Entity-Linking/blob/Professional-Structure/notebooks/Hypencoder_Vs_fine_tuned_Hypencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import importlib.util

# for syncing changes to notebook when changing something in VScode
# deprecated/need to update to new IPython which breaks colab
# %load_ext autoreload
# %autoreload 2



REPO_NAME = "Hypencoder-Entity-Linking"
GIT_URL = f"https://github.com/Steve-Falkovsky/{REPO_NAME}.git"
BRANCH_NAME = "Professional-Structure"



# --- COLAB SETUP ---
is_colab = importlib.util.find_spec("google.colab") is not None
if is_colab:
    print("☁️ Running in Colab...")
    if not os.path.exists(REPO_NAME):
        !git clone -b {BRANCH_NAME} --single-branch {GIT_URL}

    # Move into the downloaded repo (The Root)
    os.chdir(REPO_NAME)



# --- LOCAL SETUP ---
else:
    print("💻 Running Locally...")
    if os.path.basename(os.getcwd()) == "notebooks":
        os.chdir("..")


%pip install -q -e "./hypencoder-paper"

os.chdir("hypencoder-paper")

print(f"📍 Working Directory is now: {os.getcwd()}")
print("✅ Environment Ready!")

☁️ Running in Colab...
Cloning into 'Hypencoder-Entity-Linking'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 244 (delta 99), reused 193 (delta 55), pack-reused 0 (from 0)
Receiving objects: 100% (244/244), 680.53 KiB | 3.40 MiB/s, done.
Resolving deltas: 100% (99/99), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 4.2 MB/s eta 0:00:00
📍 Working Directory is now: /content/Hypencoder-Entity-Linking/hypencoder-paper
✅ Environment Ready!


In [2]:
from datasets import load_dataset

# there are all "positive" pairs"
dataset = load_dataset("Stevenf232/BC5CDR_MeSH2015_nameonly")

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

name_only_train.jsonl: 0.00B [00:00, ?B/s]

name_only_val.jsonl: 0.00B [00:00, ?B/s]

name_only_test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2654 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2559 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2656 [00:00<?, ? examples/s]

In [3]:
# take a sample of entries until I can get the batching fixed
train_pairs = dataset['train']
#train_pairs = train_pairs[:100]
print(train_pairs)
#print(len(train_pairs))

mention_names = train_pairs['mention']
entity_names = train_pairs['entity']
print(mention_names[:3])
print(entity_names[:3])
print(len(mention_names))


Dataset({
    features: ['mention', 'entity', 'id'],
    num_rows: 2654
})
['human immunodeficiency virus', "non-Hodgkin's lymphoma", 'renal cell carsinom']
['HIV Infections', 'Lymphoma, Non-Hodgkin', 'Carcinoma, Renal Cell']
2654


### Load the model

In [29]:
# Core Hypencoder model for outputing dense vector representations
from hypencoder_cb.modeling.hypencoder import Hypencoder, HypencoderDualEncoder, TextEncoder
from transformers import AutoTokenizer

model_name = "jfkback/hypencoder.2_layer"
#model_name = "Stevenf232/hypencoder_BC5CDR"

dual_encoder = HypencoderDualEncoder.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


query_encoder: Hypencoder = dual_encoder.query_encoder
passage_encoder: TextEncoder = dual_encoder.passage_encoder

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/483M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/483M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

### Move the model to the GPU

In [30]:
import torch

# Setup the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")  # This should say 'cuda'

# Move the model to the GPU
passage_encoder.to(device)
query_encoder.to(device)

Using device: cuda


Hypencoder(
  (transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

### Load datasets and tokenise

In [31]:
# convert from type "datasets" to python list
queries = list(mention_names)
passages = list(entity_names)


# the output of the tokenizer contains 3 fields:
# input_ids, token_type_ids, and attention_mask
# all contain a tensor in the shape (number of queries, max number of tokens)

query_inputs = tokenizer(queries, return_tensors="pt", padding=True, truncation=True)
passage_inputs = tokenizer(passages, return_tensors="pt", padding=True, truncation=True)


In [32]:
print(f"query_inputs:\n{query_inputs}")
print("\n\n\n")
print(f"passage_inputs:\n{passage_inputs}")

query_inputs:
{'input_ids': tensor([[  101,  2529, 10047,  ...,     0,     0,     0],
        [  101,  2512,  1011,  ...,     0,     0,     0],
        [  101, 25125,  3526,  ...,     0,     0,     0],
        ...,
        [  101, 28093,  2378,  ...,     0,     0,     0],
        [  101, 10699,  6689,  ...,     0,     0,     0],
        [  101, 26122,   102,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}




passage_inputs:
{'input_ids': tensor([[  101,  9820, 15245,  ...,     0,     0,     0],
        [  101,  1048, 24335,  ...,     0, 

# Passage Encodings


In [33]:
from tqdm import tqdm
import torch
from torch.amp import autocast

def batch_encode_passages(encoder ,passages):
  batch_size=256
  entity_name_features = []

  num_passages = passages["input_ids"].shape[0]

  with torch.no_grad(): # Disable gradient calculation (saves tons of memory)
    for i in tqdm(range(0, num_passages, batch_size), desc="Extracting features"):

        # extract entity features
        # Autocast does the math in fp16 where possible (default is fp32)
        # this will save memory and increase speed. The loss in precision shouldn't matter much (can check on a small sample if we want)
        with autocast("cuda"):
          features = encoder(
              input_ids=passages["input_ids"][i:i + batch_size].to(device),
              attention_mask=passages["attention_mask"][i:i + batch_size].to(device)
            ).representation

          entity_name_features.append(features.detach().cpu()) # Detach and move to CPU to save VRAM/RAM


  features_tensor = torch.cat(entity_name_features, dim=0)

  return features_tensor

In [34]:
passage_embeddings = batch_encode_passages(passage_encoder, passage_inputs)

Extracting features: 100%|██████████| 11/11 [00:01<00:00,  6.65it/s]


In [35]:
print(passage_embeddings.shape) # this is a tensor

# N = query_inputs["input_ids"].shape[0] # number of queries (mentions)
# M = passage_embeddings.shape[0] # number of passages (entities)
# H = passage_embeddings.shape[-1] # Hidden dimension (e.g., 768)

# # q_nets expect the shape (N, M, H)
# # so we need to reshape the passage_embeddings which are currently of shape (M, H)

# # this operation creates a 3D tensor which takes too much memory
# passage_embeddings_multi = passage_embeddings.repeat(N, 1).reshape(N, M, H)
# print(f"passage_embeddings shape: {passage_embeddings_multi.shape}")

# because we're going to batch the q-nets, we only need the number of queries we're examining at any one time:
# so instead of (N, M, H), we do (batch_size, M, H)

torch.Size([2654, 768])


# Q-nets take **a lot** of memory.

Instead of creating all of them and then doing the similarity calculation, we will create batches and calculate similarities for just those q-nets, then discard those q-nets and move on to the next batch.

In [36]:
def batch_encode_queries(encoder, queries, passage_embeddings):
  batch_size = 8
  similarity_scores = []

  num_queries = queries["input_ids"].shape[0]

  with torch.no_grad():
    for i in tqdm(range(0, num_queries, batch_size), desc="Creating q-nets and calculating similarity scores"):

        # create q-nets
        with autocast("cuda"):
          q_nets = encoder(
              input_ids=queries["input_ids"][i:i + batch_size].to(device),
              attention_mask=queries["attention_mask"][i:i + batch_size].to(device)
            ).representation


        passages_gpu = passage_embeddings.to(device)

        # q_nets expects (batch_size, M, H) -> Outputs (batch_size, M, 1)
        # .expand() creates a view, so it costs almost 0 memory! (compared to .repeat() which changes the tensor)
        # Note: we use q_nets.num_queries (out noTorch equivalent of q_nets.shape[0]) instead of batch_size
        # because the total number might not be divisible by batch_size so the last batch might be smaller than the actual batch size
        passages_batch = passages_gpu.unsqueeze(0).expand(q_nets.num_queries, -1, -1)

        # calculate similarity
        batch_scores = q_nets(passages_batch)
        similarity_scores.append(batch_scores.detach().cpu())


  scores_tensor = torch.cat(similarity_scores, dim=0)
  return scores_tensor


In [37]:
similarity_scores = batch_encode_queries(query_encoder, query_inputs, passage_embeddings)

Creating q-nets and calculating similarity scores: 100%|██████████| 332/332 [00:12<00:00, 27.27it/s]


In [38]:
# Case 1 - comparing a query to its respective passage

# In the simple case where each q_net only takes one passage, we can just
# reshape the passage_embeddings to (N, 1, H).
# passage_embeddings_single = passage_embeddings.unsqueeze(1)
# print(f"passage_embeddings shape: {passage_embeddings_single.shape}")
# giving the nueral network the input of passage_embeddings
# the output provides the relevance score of query 1 against passage 1, query 2 against passage 2, etc...
# scores = q_nets(passage_embeddings_single)
# print(f"scores: {scores}")

In [39]:
# Case 2 - comparing a query to all passages

# The case where each q_net takes multiple passages
# meaning multiple passages are now associated with each of the queries

# this operation creates a 3D tensor which takes too much memory
# passage_embeddings_multi = passage_embeddings.repeat(N, 1).reshape(N, M, H)
# print(f"passage_embeddings shape: {passage_embeddings_multi.shape}")


# unbatched similarity scores for q-nets
# similarity_scores = q_nets(passage_embeddings_multi)
# print(f"similarity_scores shape: {similarity_scores.shape}")
#print(f"similarity_scores: {similarity_scores}")


In [40]:
similarity_scores

tensor([[[-0.9448],
         [-3.7352],
         [-3.8367],
         ...,
         [-4.0552],
         [-3.9713],
         [-3.9775]],

        [[ 1.8515],
         [20.8279],
         [ 2.1806],
         ...,
         [ 1.6431],
         [ 1.6626],
         [ 1.6901]],

        [[-2.4098],
         [-2.1943],
         [13.4650],
         ...,
         [-2.3812],
         [-2.3181],
         [-2.4294]],

        ...,

        [[-1.1285],
         [-1.1976],
         [-1.2293],
         ...,
         [ 3.4296],
         [-1.3097],
         [-1.4191]],

        [[-1.2152],
         [-1.2602],
         [-1.2448],
         ...,
         [-1.3839],
         [ 5.1969],
         [-1.3562]],

        [[ 0.7650],
         [ 0.7661],
         [ 0.7462],
         ...,
         [ 0.6479],
         [ 0.5876],
         [ 0.3636]]])

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def evaluate(train_pairs):
  correct_count = 0
  top_idxs = torch.argmax(similarity_scores,dim=1).flatten()

  for i in range(len(queries)):
      top_idx = top_idxs[i]
      # the conversion to int from here on out is because the original idx is of type numpy.int64
      top_match_id = train_pairs["id"][int(top_idx)]
      correct_id = train_pairs["id"][int(i)]

      if top_match_id == correct_id:
          correct_count += 1

      mention_name = train_pairs["mention"][int(i)]
      top_match = train_pairs["entity"][int(top_idx)]
      correct_name = train_pairs["entity"][int(i)]
      print(f"mention_name: {mention_name}\ncorrect entity name: {correct_name}\ntop_match: {top_match}\n")


  print(f"total comparisons: {len(queries)}")
  print(f"correct comparisons: {correct_count}")
  print(f"accuracy: {correct_count / len(queries)}")

In [42]:
evaluate(train_pairs)

Streaming output truncated to the last 5000 lines.

mention_name: calcium chloride
correct entity name: Calcium Chloride
top_match: Calcium Chloride

mention_name: neoplasms
correct entity name: Neoplasms
top_match: Neoplasms

mention_name: streptococcal endophthalmitis
correct entity name: Streptococcal Infections
top_match: Streptococcal Infections

mention_name: cranial hemorrhage
correct entity name: Cerebral Hemorrhage
top_match: Intracranial Hemorrhages

mention_name: depressive symptoms
correct entity name: Depressive Disorder
top_match: Depressive Disorder

mention_name: hepatorenal syndrome
correct entity name: Hepatorenal Syndrome
top_match: Hepatorenal Syndrome

mention_name: Migraine
correct entity name: Migraine Disorders
top_match: Migraine Disorders

mention_name: Alopecia
correct entity name: Alopecia
top_match: Alopecia

mention_name: hypoxia
correct entity name: Anoxia
top_match: Hypoxia, Brain

mention_name: thyroxine
correct entity name: Thyroxine
top_match: Thyroxi

In [ ]:
# more evaluation methods
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# print(f"{accuracy_score(train_labels, predicted_labels)=:.3f}")
# print(f"{recall_score(train_labels, predicted_labels)=:.3f}")
# print(f"{precision_score(train_labels, predicted_labels)=:.3f}")
# print(f"{f1_score(train_labels, predicted_labels)=:.3f}")